In [12]:
# Cell 1: 魔法命令和导入
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime, timedelta

# 导入我们需要测试的核心任务
# 注意：我们直接从 .py 文件中导入，而不是通过 Prefect 的 flow.run()
from src.pipelines.data_pipeline import process_features_for_single_day

print("开发环境准备就绪！")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
开发环境准备就绪！


[autoreload of feature_repo.feature_views failed: Traceback (most recent call last):
  File "C:\Users\30630\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\30630\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
                      ^^^^^^^^
UnicodeDecodeError: 'gbk' codec can't decode byte 0x81 in position 277: illegal multibyte sequence
]
[autoreload of src.pipelines.data_pipeline failed: Traceback (most recent call last):
  File "C:\Users\30630\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\30630\AppData\Roaming\Python

In [ ]:
# Cell 2: 初始化 Spark Session (使用 Spark Connect)
# 这部分代码会连接到你远程的 K8s Spark 集群
from pyspark.sql import SparkSession

def get_spark_session() -> SparkSession:
    """获取或创建一个 SparkSession。"""
    # return SparkSession.builder.appName("FeatureETL-MovieLens").getOrCreate()

    return SparkSession.builder.remote("sc://localhost:31002").getOrCreate()
print(f"成功连接到 Spark 集群，版本: {spark.version}")

# 如果你用本地 Spark，则是:
# spark = SparkSession.builder.master("local[*]").appName("LocalDev").getOrCreate()



成功连接到 Spark 集群，版本: 4.0.0


In [10]:
spark.sql(f"DESCRIBE FORMATTED raw_data.movielens_ratings").show(truncate=False)


+----------------------------+---------------------------------------------------+-------+
|col_name                    |data_type                                          |comment|
+----------------------------+---------------------------------------------------+-------+
|userId                      |int                                                |NULL   |
|movieId                     |int                                                |NULL   |
|rating                      |float                                              |NULL   |
|timestamp                   |bigint                                             |NULL   |
|rating_date                 |date                                               |NULL   |
|# Partition Information     |                                                   |       |
|# col_name                  |data_type                                          |comment|
|rating_date                 |date                                               |NULL   |

In [11]:
# Cell 3: 运行你的核心任务进行测试
# 我们在这里手动调用 process_features_for_single_day 函数
# 它会使用我们刚刚连接好的 Spark 集群来执行
try:
    print("开始测试 process_features_for_single_day...")
    # 模拟一个测试日期
    test_date = (datetime.now() - timedelta(days=50)).strftime('%Y-%m-%d')
    test_date = "2019-11-01"
    
    # 直接调用函数，绕过了 Prefect 的 Task Runner
    # 这样可以快速看到 Spark 的执行日志和任何报错
    process_features_for_single_day.fn(
        target_date_str=test_date,
        feature_groups_to_process=["user_rolling_ratings"]
    )
    
    print("\n 函数执行成功！")

except Exception as e:
    print(f"\n 函数执行失败: {e}")
    # 在这里你可以进行断点调试



开始测试 process_features_for_single_day...

 函数执行失败: [CONNECT_URL_NOT_SET] Cannot create a Spark Connect session because the Spark Connect remote URL has not been set. Please define the remote URL by setting either the 'spark.remote' option or the 'SPARK_REMOTE' environment variable.


In [ ]:
# Cell 4: 验证结果
# 函数执行成功后，你可以在这里用 Spark SQL 来检查输出的 Hive 表是否符合预期
print("\n验证输出数据...")
output_table = "feature_store.user_rolling_ratings_historical"
result_df = spark.sql(f"""
    SELECT * FROM {output_table} 
    WHERE timestamp = to_date('{test_date}', 'yyyy-MM-dd') 
    LIMIT 10
""")

result_df.show()

spark.stop()